<a href="https://colab.research.google.com/github/argonne-lcf/ai-science-training-series/blob/main/05_llm_part2/Intro_to_Prompt_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW5
Madusanka Madiligama 


In [1]:
import os
from getpass import getpass
os.environ['HUGGINGFACEHUB_API_TOKEN'] = getpass('Enter huggingfacehub api token: ')

Enter huggingfacehub api token:  ········


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

# load model
model = "tiiuae/falcon-7b-instruct"
# model = "tiiuae/falcon-40b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model)

falcon_pipeline = transformers.pipeline("text-generation",
                                        model=model,
                                        tokenizer=tokenizer,
                                        torch_dtype=torch.bfloat16,
                                        trust_remote_code=True,
                                        device_map="auto"
                                        )

/soft/applications/conda/2024-08-08/mconda3/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
2024-11-18 18:54:05.923950: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731956046.131732 3711848 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731956046.209242 3711848 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-18 18:

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
# load remote model
from langchain.llms import HuggingFaceHub
model = "tiiuae/falcon-7b-instruct"
# model = "tiiuae/falcon-40b-instruct"
falcon = HuggingFaceHub(
    repo_id=model,
    model_kwargs={"temperature": 0.5,
                  "max_length": 128},
)

/var/tmp/pbs.37527.sophia-pbs-01.lab.alcf.anl.gov/ipykernel_3711848/454465620.py:5: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  falcon = HuggingFaceHub(


In [4]:
def get_completion_falcon(input, host="remote", **kwargs):
    prompt = f"#### User: \n{input}\n\n#### Response from falcon-7b-instruct:"
    response = ""
    # print(prompt)
    if host.lower() == "local":
      print("invoking llm at Google Colab")
      if 'max_length' not in kwargs:
        kwargs['max_length'] = 1000

      falcon_response = falcon_pipeline(prompt,
                                      #max_length=500,
                                      do_sample=True,
                                      top_k=10,
                                      num_return_sequences=1,
                                      eos_token_id=tokenizer.eos_token_id,
                                      **kwargs,
                                      )
      response = falcon_response[0]['generated_text']

    elif host.lower() == "remote":
      print("invoking llm at Huggingface Hub")
      if "max_length" in kwargs:
        kwargs['max_new_tokens'] = kwargs['max_length']

      response = falcon.invoke(prompt, **kwargs)

    else:
      print ("invalid host value, must be 'remote' or 'local'")

    return response

### Homework

1. Load in a generative model using the HuggingFace pipeline. Use the zero-shot, few-shot, chain-of-thought, and few-shot chain-of-thought prompting to get the sum of odd numbers from a list of integers. In a few sentences describe what you learnt from each approach of prompting.
- Next, play around with the temperature parameter. In a few sentences describe what you changes you notice.


In [14]:
kwargs = {"max_length": 1000}
zero_shot_prompt = """

You are a mathematical genius.

Answer the following question directly.

Q: Do the odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1?

A:

"""
zero_shot_prompt_explanation = get_completion_falcon(zero_shot_prompt, **kwargs)
print(zero_shot_prompt_explanation)


invoking llm at Huggingface Hub
#### User: 


You are a mathematical genius.

Answer the following question directly.

Q: Do the odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1?

A:



#### Response from falcon-7b-instruct:

Yes, the odd numbers in this group add up to an even number. The odd numbers are 1, 5, 13, and 82. The even numbers are 32 and 7. Therefore, the sum of the odd numbers is 13 and the sum of the even numbers is 32.


In [15]:
kwargs = {"max_length": 1000}
few_shot_prompt = """

You are a mathematical genius.

Answer the questions using the following examples without showing detailed reasoning.

Examples:

Q: The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.

A: False

Q: The odd numbers in this group add up to an even number: 17, 10, 19, 4, 8, 12, 24.

A: True

Q: The odd numbers in this group add up to an even number: 16, 11, 14, 4, 8, 13, 24.

A: True

Q: The odd numbers in this group add up to an even number: 17, 9, 10, 12, 13, 4, 2.

A: False

Problem:

Q: The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.

A:

"""
few_shot_prompt_explanation = get_completion_falcon(few_shot_prompt, **kwargs)
print(few_shot_prompt_explanation)

invoking llm at Huggingface Hub
#### User: 


You are a mathematical genius.

Answer the questions using the following examples without showing detailed reasoning.

Examples:

Q: The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.

A: False

Q: The odd numbers in this group add up to an even number: 17, 10, 19, 4, 8, 12, 24.

A: True

Q: The odd numbers in this group add up to an even number: 16, 11, 14, 4, 8, 13, 24.

A: True

Q: The odd numbers in this group add up to an even number: 17, 9, 10, 12, 13, 4, 2.

A: False

Problem:

Q: The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.

A:



#### Response from falcon-7b-instruct:

Yes, the odd numbers in this group add up to an even number. The odd numbers 15 and 32, add up to an even number 32. The odd numbers 5 and 13, add up to an even number 13. The odd numbers 82 and 7, add up to an even number 82. The odd numbers 13 and 82, add up to an even number 82. The odd numbers 7 and 

In [16]:
COT_prompt = """

You are a mathematical genius.

Provide a detailed explanation for the answer to the question.

Q: The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.

A: The given numbers are: (15, 32, 5, 13, 82, 7, 1). First, identify the odd numbers within the list: (15, 5, 13, 7, 1). Next, add these odd numbers together: 15 + 5 + 13 + 7 + 1 equals 41. Since 41 is an odd number, the odd numbers do not add up to an even number, hence the answer is False.

"""
COT_prompt_explanation = get_completion_falcon(COT_prompt, **kwargs)

print(COT_prompt_explanation)

invoking llm at Huggingface Hub
#### User: 


You are a mathematical genius.

Provide a detailed explanation for the answer to the question.

Q: The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.

A: The given numbers are: (15, 32, 5, 13, 82, 7, 1). First, identify the odd numbers within the list: (15, 5, 13, 7, 1). Next, add these odd numbers together: 15 + 5 + 13 + 7 + 1 equals 41. Since 41 is an odd number, the odd numbers do not add up to an even number, hence the answer is False.



#### Response from falcon-7b-instruct:

The odd numbers in the given group add up to an even number: 15, 32, 5, 13, 82, 7, 1. This is a very simple problem, but it is a bit tricky. The odd numbers add up to an even number, so the answer is False.


In [17]:
few_shot_COT_prompt =  """You are a mathematical genius.
The template of the question is given as Q: [question]. The template of the answer is given as A: [answer].
Help me solve the problem below using the examples given.

Examples:
Q: The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: The given numbers are: (4, 8, 9, 15, 12, 2, 1). Next select the odd numbers (9, 15, 1).  Adding all the selected odd numbers (9, 15, 1) gives 25. 25 is an odd number, hence the answer is False.

Q: The odd numbers in this group add up to an even number: 17, 10, 19, 4, 8, 12, 24.
A: The given numbers are: (17, 10, 19, 4, 8, 12, 24). Next select the odd numbers (17, 19).  Adding all the selected odd numbers (17, 19) gives 36. 36 is an even number, hence the answer is True.

Q: The odd numbers in this group add up to an even number: 16, 11, 14, 4, 8, 13, 24.
A: The given numbers are: (16, 11, 14, 4, 8, 13, 24). Next select the odd numbers (11, 13).  Adding all the selected odd numbers (11, 13) gives 24. 24 is an even number, hence the answer is True.

Q: The odd numbers in this group add up to an even number: 17, 9, 10, 12, 13, 4, 2.
A: The given numbers are: (17, 9, 10, 12, 13, 4, 2). Next select the odd numbers (17, 9, 13).  Adding all the selected odd numbers (17, 9, 13) gives 39. 39 is an odd number, hence the answer is False.

Problem:
Q: The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.
A:
"""
#15 + 5 + 13 + 7 + 1 = 41
few_shot_COT_prompt_explanation = get_completion_falcon(few_shot_COT_prompt, **kwargs)
print(few_shot_COT_prompt_explanation)

invoking llm at Huggingface Hub
#### User: 
You are a mathematical genius.
The template of the question is given as Q: [question]. The template of the answer is given as A: [answer].
Help me solve the problem below using the examples given.

Examples:
Q: The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: The given numbers are: (4, 8, 9, 15, 12, 2, 1). Next select the odd numbers (9, 15, 1).  Adding all the selected odd numbers (9, 15, 1) gives 25. 25 is an odd number, hence the answer is False.

Q: The odd numbers in this group add up to an even number: 17, 10, 19, 4, 8, 12, 24.
A: The given numbers are: (17, 10, 19, 4, 8, 12, 24). Next select the odd numbers (17, 19).  Adding all the selected odd numbers (17, 19) gives 36. 36 is an even number, hence the answer is True.

Q: The odd numbers in this group add up to an even number: 16, 11, 14, 4, 8, 13, 24.
A: The given numbers are: (16, 11, 14, 4, 8, 13, 24). Next select the odd numbers (11, 13).  Adding a

In [18]:
temperature_value = 0.1  # Adjust the temperature value as needed between 0 and 1
few_shot_COT_prompt_explanation = get_completion_falcon(few_shot_COT_prompt, temperature=temperature_value)

print(few_shot_COT_prompt_explanation)



invoking llm at Huggingface Hub
#### User: 
You are a mathematical genius.
The template of the question is given as Q: [question]. The template of the answer is given as A: [answer].
Help me solve the problem below using the examples given.

Examples:
Q: The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: The given numbers are: (4, 8, 9, 15, 12, 2, 1). Next select the odd numbers (9, 15, 1).  Adding all the selected odd numbers (9, 15, 1) gives 25. 25 is an odd number, hence the answer is False.

Q: The odd numbers in this group add up to an even number: 17, 10, 19, 4, 8, 12, 24.
A: The given numbers are: (17, 10, 19, 4, 8, 12, 24). Next select the odd numbers (17, 19).  Adding all the selected odd numbers (17, 19) gives 36. 36 is an even number, hence the answer is True.

Q: The odd numbers in this group add up to an even number: 16, 11, 14, 4, 8, 13, 24.
A: The given numbers are: (16, 11, 14, 4, 8, 13, 24). Next select the odd numbers (11, 13).  Adding a

In [19]:
temperature_value = 0.5  # Adjust the temperature value as needed between 0 and 1
few_shot_COT_prompt_explanation_t5 = get_completion_falcon(few_shot_COT_prompt, temperature=temperature_value)

print(few_shot_COT_prompt_explanation_t5)

invoking llm at Huggingface Hub
#### User: 
You are a mathematical genius.
The template of the question is given as Q: [question]. The template of the answer is given as A: [answer].
Help me solve the problem below using the examples given.

Examples:
Q: The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: The given numbers are: (4, 8, 9, 15, 12, 2, 1). Next select the odd numbers (9, 15, 1).  Adding all the selected odd numbers (9, 15, 1) gives 25. 25 is an odd number, hence the answer is False.

Q: The odd numbers in this group add up to an even number: 17, 10, 19, 4, 8, 12, 24.
A: The given numbers are: (17, 10, 19, 4, 8, 12, 24). Next select the odd numbers (17, 19).  Adding all the selected odd numbers (17, 19) gives 36. 36 is an even number, hence the answer is True.

Q: The odd numbers in this group add up to an even number: 16, 11, 14, 4, 8, 13, 24.
A: The given numbers are: (16, 11, 14, 4, 8, 13, 24). Next select the odd numbers (11, 13).  Adding a

In [20]:
temperature_value = 0.9  # Adjust the temperature value as needed between 0 and 1
few_shot_COT_prompt_explanation_t9 = get_completion_falcon(few_shot_COT_prompt, temperature=temperature_value)

print(few_shot_COT_prompt_explanation_t9)

invoking llm at Huggingface Hub
#### User: 
You are a mathematical genius.
The template of the question is given as Q: [question]. The template of the answer is given as A: [answer].
Help me solve the problem below using the examples given.

Examples:
Q: The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: The given numbers are: (4, 8, 9, 15, 12, 2, 1). Next select the odd numbers (9, 15, 1).  Adding all the selected odd numbers (9, 15, 1) gives 25. 25 is an odd number, hence the answer is False.

Q: The odd numbers in this group add up to an even number: 17, 10, 19, 4, 8, 12, 24.
A: The given numbers are: (17, 10, 19, 4, 8, 12, 24). Next select the odd numbers (17, 19).  Adding all the selected odd numbers (17, 19) gives 36. 36 is an even number, hence the answer is True.

Q: The odd numbers in this group add up to an even number: 16, 11, 14, 4, 8, 13, 24.
A: The given numbers are: (16, 11, 14, 4, 8, 13, 24). Next select the odd numbers (11, 13).  Adding a

The varying temperature settings in the Falcon-7b-instruct model responses illustrate distinct impacts on output style and accuracy.​ At a low temperature of 0.1, the answers are conservative and deterministic, leading to straightforward conclusions like "False," yet containing errors in identifying odd numbers due to a minimal creative approach. With a moderate temperature of 0.5, the output strikes a balance between coherence and creativity, repeating similar selection errors but demonstrating more detailed step-by-step engagement. At a high temperature of 0.9, the responses exhibit increased randomness and creative alterations, resulting in more diverse outputs but with a sacrifice in logical coherence and accuracy, highlighting the trade-off between creativity and factual correctness.